<a href="https://colab.research.google.com/github/cwyalpha/Langchain-Chatchat-Colab/blob/main/Langchain_Chatchat_ChatGLM2_6b_int4_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 拉取仓库
!git clone https://github.com/THUDM/ChatGLM2-6B
%cd /content/ChatGLM2-6B
# int4版本需要安装其中的cpm_kernels
!pip install -r requirements.txt

%cd /content/
!git clone https://github.com/imClumsyPanda/langchain-ChatGLM.git
%cd /content/langchain-ChatGLM/
!pip install -r requirements.txt
!pip install --upgrade protobuf==3.19.6
!pip install gradio
!pip install accelerate
!pip install spacy
!pip install streamlit
!pip install pyngrok


#Download models
!git lfs install
%cd /content/langchain-ChatGLM
!git clone https://huggingface.co/THUDM/chatglm2-6b-int4
!git clone https://huggingface.co/moka-ai/m3e-base

#复制配置
%cp /content/langchain-ChatGLM/configs/model_config.py.example /content/langchain-ChatGLM/configs/model_config.py
%cp /content/langchain-ChatGLM/configs/server_config.py.example /content/langchain-ChatGLM/configs/server_config.py
#修改model_config.py文件
!sed -i "s/THUDM\/chatglm2-6b\"/\/content\/langchain-ChatGLM\/chatglm2-6b-int4\"/g" /content/langchain-ChatGLM/configs/model_config.py
!sed -i "s/moka-ai\/m3e-base/\/content\/langchain-ChatGLM\/m3e-base/g" /content/langchain-ChatGLM/configs/model_config.py


%cd /content/langchain-ChatGLM


In [ ]:
# Runtime里类型要改成GPU
import torch
EMBEDDING_DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# 确认输出是cuda
print(EMBEDDING_DEVICE)

In [ ]:
!python init_database.py --recreate-vs
!nohup python server/llm_api.py &

In [ ]:
# 稍等上一步跑完再执行
!nohup python server/api.py &

In [ ]:
!streamlit run webui.py &>/dev/null&

In [7]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!npm install localtunnel
#!streamlit run app.py &>web_logs.txt &
!npx localtunnel --port 8501
# 在网页中输入IP后提交

Password/Enpoint IP for localtunnel is: 34.91.53.228
npm WARN saveError ENOENT: no such file or directory, open '/content/langchain-ChatGLM/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/langchain-ChatGLM/package.json'
npm WARN langchain-ChatGLM No description
npm WARN langchain-ChatGLM No repository field.
npm WARN langchain-ChatGLM No README data
npm WARN langchain-ChatGLM No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 1.279s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

npx: installed 22 in 2.259s
your url is: https://legal-poets-add.loca.lt
^C
